<a href="https://colab.research.google.com/github/Anunineleaps6/Capstone_Sales/blob/main/Capstone_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gspread oauth2client pandas numpy


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
client = gspread.authorize(creds)


In [6]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [7]:
!pip install phonenumbers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.5 MB/s eta 0:00:00


In [8]:
import phonenumbers

def generate_phone(country, region_mapping):
    # Try to get the country code from the region_mapping first
    code = None
    for region, data in region_mapping.items():
        if country in data["countries"]:
            code = data["dial_code"].get(country)
            break

    if code:
        try:
            # Generate a random 10-digit number
            local_number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            # Attempt to parse and format using phonenumbers
            # Note: This might not create *valid* numbers, but will follow the format
            # We need a country code without the '+' for the parse function
            country_code_for_parse = code.lstrip('+')
            # We need to find a region code (like US, GB, IN) for the parse function
            # This is a simplified approach; a more robust solution would map countries to region codes
            country_region_code = ""
            for region_key, data in region_mapping.items():
                if country in data.get("countries", []):
                     # This is a simplification; need a proper country to region code mapping
                     # For now, we can try to infer from common country codes or use a default
                     # A more accurate approach would involve a comprehensive country code to region code mapping
                     # Let's use a placeholder logic or rely on the dial code if a direct mapping is complex
                     pass # Will refine this

            # A better approach might be to directly format a random number with the known dial code
            # We'll stick to generating a random 10-digit number and prefixing the dial code
            # as phonenumbers.parse requires a known region or a leading '+' which we already have

            # Let's refine the phonenumbers usage or simplify if direct generation is sufficient
            # Given the user wants the format "+[country_code]-[10_digits]", direct generation is simpler
            number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            return f"{code}-{number}"

        except Exception as e:
            print(f"Error generating phone for {country} with code {code}: {e}")
            # Fallback to a generic format if phonenumbers fails
            return f"{code}-{random.randint(1000000000, 9999999999)}"
    else:
        # Fallback for countries not in region_mapping
        return f"+00-{random.randint(1000000000, 9999999999)}"

# Now, modify the function call in the lead generation loop in cell 9212e2f3
# (This modification will be done in the next modify_cells command)

In [9]:
#LEADS
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta

# ------------------------------------------
# 1. Region, Country, Dial Code Mapping (Used for phone number generation)
# ------------------------------------------
region_mapping = {
    "NA": {"countries": ["USA", "Canada"], "dial_code": {"USA": "+1", "Canada": "+1"}, "industries": ["Technology", "Healthcare", "Finance", "Retail"], "services": ["Data", "Marketing", "Cloud"]},
    "EU": {"countries": ["UK", "Germany", "France", "Netherlands", "Switzerland", "Sweden", "Israel"], "dial_code": {"UK": "+44", "Germany": "+49", "France": "+33", "Netherlands": "+31", "Switzerland": "+41", "Sweden": "+46", "Israel": "+972"}, "industries": ["Automotive", "Pharmaceuticals", "Banking", "Consulting"], "services": ["Consulting", "Engineering", "Operations"]},
    "APAC": {"countries": ["India", "Japan", "Singapore", "China", "Indonesia", "Australia", "New Zealand", "S. Korea"], "dial_code": {"India": "+91", "Japan": "+81", "Singapore": "+65", "China": "+86", "Indonesia": "+62", "Australia": "+61", "New Zealand": "+64", "S. Korea": "+82"}, "industries": ["Technology", "E-Commerce", "Manufacturing", "Finance"], "services": ["AI", "Staffing", "T&M"]},
    "ME": {"countries": ["UAE", "Saudi Arabia", "Qatar", "Bahrain"], "dial_code": {"UAE": "+971", "Saudi Arabia": "+966", "Qatar": "+974", "Bahrain": "+973"}, "industries": ["Energy", "Real Estate", "Banking"], "services": ["Data", "Consulting"]},
    "LA": {"countries": ["Brazil", "Mexico", "Argentina", "Chile", "Colombia", "Peru"], "dial_code": {"Brazil": "+55", "Mexico": "+52", "Argentina": "+54", "Chile": "+56", "Colombia": "+57", "Peru": "+51"}, "industries": ["Manufacturing", "Agriculture", "Energy"], "services": ["Operations", "Logistics"]},
    "AF": {"countries": ["S. Africa", "Mozambique", "Kenya", "Morocco"], "dial_code": {"S. Africa": "+27", "Mozambique": "+258", "Kenya": "+254", "Morocco": "+212"}, "industries": ["Telecommunications", "Mining", "Finance"], "services": ["Staffing", "Service"]}
}

# ------------------------------------------
# 2. Predefined Lead Data (for Company_Name, Country, Region, Lead_ID) - 108 Companies
# ------------------------------------------
predefined_leads = [
    # North America (20)
    ("Uber", "USA", "NA", "LDUBR-NA-001"), ("Walmart", "USA", "NA", "LDWMT-NA-002"), ("Ford Motor Co", "USA", "NA", "LDFRD-NA-003"), ("Amazon", "USA", "NA", "LDAMZ-NA-004"),
    ("Microsoft", "USA", "NA", "LDMST-NA-005"), ("Target", "USA", "NA", "LDTGT-NA-006"), ("Uber Eats", "USA", "NA", "LDUBE-NA-007"), ("CVS Health", "USA", "NA", "LDCVS-NA-008"),
    ("McKesson", "USA", "NA", "LDMCK-NA-009"), ("TD Bank Group", "Canada", "NA", "LDTDB-NA-010"), ("Shopify", "Canada", "NA", "LDSHF-NA-011"), ("Suncor Energy", "Canada", "NA", "LDSCR-NA-012"),
    ("Kaiser Permanente", "USA", "NA", "LDKPR-NA-013"), ("Wells Fargo", "USA", "NA", "LDWLF-NA-014"), ("Canadian Tire", "Canada", "NA", "LDCTC-NA-015"), ("American Tower", "USA", "NA", "LDAMT-NA-016"),
    ("Tim Hortons", "Canada", "NA", "LDTHC-NA-017"), ("Lowe's", "USA", "NA", "LDLWS-NA-018"), ("PicnicHealth", "USA", "NA", "LDPCH-NA-019"), ("Credit Saison India (USA operations)", "USA", "NA", "LDCSI-NA-020"),

    # Europe (16)
    ("Tesco", "UK", "EU", "LDTSCO-EU-021"), ("Lufthansa Group", "Germany", "EU", "LDLFT-EU-022"), ("Allianz Direct", "Germany", "EU", "LDALZ-EU-023"), ("HSBC", "UK", "EU", "LDHSBC-EU-024"),
    ("Sage Group", "UK", "EU", "LDSGE-EU-025"), ("Schneider Electric", "France", "EU", "LDSNE-EU-026"), ("Philips Healthcare", "Netherlands", "EU", "LDPHL-EU-027"), ("Vodafone", "UK", "EU", "LDVDF-EU-028"),
    ("Siemens", "Germany", "EU", "LDSMN-EU-029"), ("Nestlé", "Switzerland", "EU", "LDNST-EU-030"), ("ING Group", "Netherlands", "EU", "LDING-EU-031"), ("ABB", "Switzerland", "EU", "LDABB-EU-032"),
    ("Volvo Group", "Sweden", "EU", "LDVLV-EU-033"), ("Carrefour", "France", "EU", "LDCFR-EU-034"), ("ZF Group", "Germany", "EU", "LDZFG-EU-035"), ("Trigo", "UK", "EU", "LDTRG-EU-036"), # Assuming UK operations for Trigo

    # Asia-Pacific (14)
    ("DBS Bank", "Singapore", "APAC", "LDDBS-APAC-037"), ("Kiwibank", "New Zealand", "APAC", "LDKWB-APAC-038"), ("Reliance Retail", "India", "APAC", "LDRLR-APAC-039"), ("Tata Consultancy Services", "India", "APAC", "LDTCS-APAC-040"),
    ("SoftBank", "Japan", "APAC", "LDSFB-APAC-041"), ("Tokopedia / GoTo", "Indonesia", "APAC", "LDTKP-APAC-042"), ("Telstra", "Australia", "APAC", "LDTSL-APAC-043"), ("Westpac", "Australia", "APAC", "LDWSP-APAC-044"),
    ("infosys", "India", "APAC", "LDINF-APAC-045"), ("Grab", "Singapore", "APAC", "LDGRB-APAC-046"), ("JioMart", "India", "APAC", "LDJMR-APAC-047"), ("Toyota", "Japan", "APAC", "LDTYT-APAC-048"),
    ("Sony", "Japan", "APAC", "LDSNY-APAC-049"), ("Samsung", "S. Korea", "APAC", "LDSMS-APAC-050"),

    # Middle East (4)
    ("Saudi Aramco", "Saudi Arabia", "ME", "LDSAR-ME-051"), ("ADNOC", "UAE", "ME", "LDADC-ME-052"), ("QatarEnergy", "Qatar", "ME", "LDQTE-ME-053"), ("Investcorp", "Bahrain", "ME", "LDINV-ME-054"),

    # Latin America (10)
    ("Petrobras", "Brazil", "LA", "LDPBR-LA-055"), ("América Móvil", "Mexico", "LA", "LDAMX-LA-056"), ("MercadoLibre", "Argentina", "LA", "LDMCL-LA-057"), ("Codelco", "Chile", "LA", "LDCDL-LA-058"),
    ("Ecopetrol", "Colombia", "LA", "LDECP-LA-059"), ("Latam Airlines", "Chile", "LA", "LDLTM-LA-060"), ("Natura", "Brazil", "LA", "LDNTR-LA-061"), ("Itau Unibanco", "Brazil", "LA", "LDITU-LA-062"),
    ("Bancolombia", "Colombia", "LA", "LDBCL-LA-063"), ("Grupo Mexico", "Mexico", "LA", "LDGPM-LA-064"),

    # Africa (12)
    ("MTN Group", "S. Africa", "AF", "LDMTN-AF-065"), ("Sasol", "S. Africa", "AF", "LDSAL-AF-066"), ("Standard Bank Group", "S. Africa", "AF", "LDSBG-AF-067"), ("Vodacom", "S. Africa", "AF", "LDVDC-AF-068"),
    ("Safaricom", "Kenya", "AF", "LDSFC-AF-069"), ("KCB Group", "Kenya", "AF", "LDKCB-AF-070"), ("Attijariwafa Bank", "Morocco", "AF", "LDATJ-AF-071"), ("FirstRand", "S. Africa", "AF", "LDFRD-AF-072"),
    ("Absa Group", "S. Africa", "AF", "LDABG-AF-073"), ("Nedbank Group", "S. Africa", "AF", "LDNDB-AF-074"), ("Equity Group Holdings", "Kenya", "AF", "LDEGH-AF-075"), ("Telkom", "S. Africa", "AF", "LDTLK-AF-076"),

    # Adding more predefined companies to reach 108, maintaining the pattern and region assignment
    ("Infosys", "India", "APAC", "LDINF-APAC-077"), ("Wipro", "India", "APAC", "LDWIP-APAC-078"), ("HCL Technologies", "India", "APAC", "LDHCL-APAC-079"),
    ("Tech Mahindra", "India", "APAC", "LDTMH-APAC-080"), ("Larsen & Toubro Infotech", "India", "APAC", "LDLTI-APAC-081"), ("Mindtree", "India", "APAC", "LDMNT-APAC-082"),
    ("Mphasis", "India", "APAC", "LDMPH-APAC-083"), ("L&T Technology Services", "India", "APAC", "LDLTTS-APAC-084"), ("Persistent Systems", "India", "APAC", "LDPST-APAC-085"),
    ("Coforge", "India", "APAC", "LDCFG-APAC-086"), ("Zensar Technologies", "India", "APAC", "LDZNR-APAC-087"), ("Firstsource Solutions", "India", "APAC", "LDFRS-APAC-088"),
    ("ExlService Holdings", "India", "APAC", "LDEXL-APAC-089"), ("Genus Power Infrastructures", "India", "APAC", "LDGPI-APAC-090"), ("Cyient", "India", "APAC", "LDCYI-APAC-091"),

    ("Tata Motors", "India", "APAC", "LDTMO-APAC-092"), ("Mahindra & Mahindra", "India", "APAC", "LDMAM-APAC-093"), ("Maruti Suzuki India", "India", "APAC", "LDMSI-APAC-094"),
    ("Hyundai Motor India", "India", "APAC", "LDHMI-APAC-095"), ("Kia India", "India", "APAC", "LDKIA-APAC-096"), ("Toyota Kirloskar Motor", "India", "APAC", "LDTKM-APAC-097"),
    ("Honda Cars India", "India", "APAC", "LDHCI-APAC-098"), ("Renault India", "India", "APAC", "LDRNI-099"), ("Nissan Motor India", "India", "APAC", "LDNMI-APAC-100"),

    ("Apple", "USA", "NA", "LDAPL-NA-101"), ("Google", "USA", "NA", "LDGOO-NA-102"), ("Facebook", "USA", "NA", "LDFAB-NA-103"), ("Netflix", "USA", "NA", "LDNFL-NA-104"),
    ("Salesforce", "USA", "NA", "LDSFC-NA-105"), ("Tesla", "USA", "NA", "LDTES-NA-106"), ("IBM", "USA", "NA", "LDIBM-NA-107"), ("Intel", "USA", "NA", "LDINT-NA-108")
]

num_predefined_leads = len(predefined_leads) # Define num_predefined_leads here

# ------------------------------------------
# 3. Generate Sales Reps (Unevenly distributed based on region size)
# ------------------------------------------
rep_names = [
    "Alice Smith", "Bob Johnson", "Charlie Lee", "Diana King", "Edward Brown",
    "Fiona Green", "George White", "Hannah Black", "Ian Gray", "Jill Lewis",
    "Kevin Martin", "Laura Scott", "Mike Turner", "Nina Adams", "Oscar Clark",
    "Paula Young", "Quinn Hall", "Rachel Allen", "Steve Wright", "Tina Baker"
] # 20 names

# Distribute 20 reps unevenly across regions (adjust numbers as needed to total 20)
# Example: NA: 4, EU: 4, APAC: 5, ME: 2, LA: 3, AF: 2
region_rep_distribution = {"NA": 4, "EU": 4, "APAC": 5, "ME": 2, "LA": 3, "AF": 2}

rep_list = []
rep_name_index = 0
for region, count in region_rep_distribution.items():
    for i in range(count):
        full_name = rep_names[rep_name_index]
        initials = "".join([part[0] for part in full_name.split()]).upper()
        rep_id = f"{initials}_{region}"
        rep_list.append({"rep_id": rep_id, "rep_name": full_name, "region": region})
        rep_name_index += 1

# Convert to DataFrame
rep_df = pd.DataFrame(rep_list)

# ------------------------------------------
# 4. Generate Leads DataFrame
# ------------------------------------------
def get_company_email(name):
    # Basic email domain generation
    domain = name.lower().replace(" ", "").replace("&", "").replace(".", "").replace("/", "").replace("(", "").replace(")", "") + ".com"
    return domain

# Use the generate_phone function defined in the previous cell (ad0fa588)

def generate_timestamp(start_days_ago=730): # Up to 2 years ago
    created = datetime.now() - timedelta(days=random.randint(0, start_days_ago), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400))

    # Calculate minimum required duration based on stages
    # This is a placeholder; the actual required duration depends on the final stage,
    # which is assigned later. We will ensure last_updated is after created with a buffer.
    min_buffer_days = 1 # Ensure at least one day difference if a lead progresses

    # Generate last_updated ensuring it's after created with a buffer
    last_updated = created + timedelta(days=random.randint(min_buffer_days, 60), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400)) # Ensure updated is after created with a reasonable range

    return created.strftime("%Y-%m-%d %H:%M:%S"), last_updated.strftime("%Y-%m-%d %H:%M:%S")

def get_company_initials(name):
    return "".join([w[0] for w in name.split()]).upper()

# Define service categories and industries based on region_mapping for random lead generation
all_service_categories = list(set([service for region_data in region_mapping.values() for service in region_data.get('services', [])]))
all_industries = list(set([industry for region_data in region_mapping.values() for industry in region_data.get('industries', [])]))


# Generate a list of more descriptive company name parts for random leads
company_name_parts = {
    "adjectives": ["Global", "Strategic", "Innovative", "Dynamic", "Future", "Leading", "Creative", "Advanced"],
    "nouns": ["Solutions", "Systems", "Analytics", "Consulting", "Tech", "Ventures", "Partners", "Industries"]
}

def generate_random_company_name(adjectives, nouns):
    return f"{random.choice(adjectives)}{random.choice(nouns)}"

def get_abbreviated_company_name(name):
    # Simple abbreviation logic
    words = name.split()
    if len(words) > 1:
        return "".join([w[:2] for w in words]).upper() # Use first two letters of each word
    else:
        return name[:4].upper() # Use first four letters if single word


leads_data = []

# Define stages in the desired funnel order
funnel_stages = [
    "Lead Captured",
    "Qualified",
    "Contacted",
    "Responded",
    "Demo Scheduled",
    "Demo",
    "Demo Completed",
    "Proposal Sent",
    "Proposal",
    "Negotiation"
]

# Define stages where a deal could potentially be lost *before* Negotiation
potential_early_loss_stages = ["Demo", "Demo Completed", "Proposal Sent", "Proposal"]
# Define criteria for losing a deal at earlier stages
early_loss_criteria = {
    "Demo": {"engagement_level": ["Low"], "random_chance": 0.4}, # 40% chance of losing at Demo if engagement is Low
    "Demo Completed": {"budget_tier": ["20-50L"], "engagement_level": ["Low"], "random_chance": 0.3}, # 30% chance after Demo Completed if budget low and engagement low
    "Proposal": {"engagement_level": ["Low", "Medium"], "priority": ["Low"], "random_chance": 0.5} # 50% chance at Proposal if engagement low/medium and priority low
}
# Define criteria for losing a deal after Negotiation
negotiation_loss_criteria = {"budget_tier": ["20-50L"], "engagement_level": ["Low"], "random_chance": 0.6} # 60% chance after Negotiation if budget low and engagement low

# Define the percentage of leads to be "ongoing" and stuck in intermediate stages
# Updated ongoing_percentage and distribution
ongoing_percentage = 0.30
total_leads_to_generate = 2000 # Updated total leads
num_ongoing_leads_to_generate = int(total_leads_to_generate * ongoing_percentage)

# Define the desired distribution of ongoing leads across intermediate stages
# The keys should match the stages in funnel_stages
ongoing_stage_distribution = {
    "Lead Captured": 0.031,
    "Qualified": 0.031,
    "Contacted": 0.044,
    "Responded": 0.019,
    "Demo Scheduled": 0.025,
    "Demo": 0.006,
    "Demo Completed": 0.031,
    "Proposal Sent": 0.038,
    "Proposal": 0.044,
    "Negotiation": 0.031
}

# Create a list of stages based on the desired distribution for random selection
ongoing_stages_list = []
# Scale the distribution percentages by the number of ongoing leads to get counts
total_distribution_percentage = sum(ongoing_stage_distribution.values())
# Adjust scaling factor to ensure the sum of counts is close to num_ongoing_leads_to_generate
scaling_factor = num_ongoing_leads_to_generate / total_distribution_percentage if total_distribution_percentage > 0 else 0

for stage, percentage in ongoing_stage_distribution.items():
    # Add the stage name to the list based on the calculated count
    num_to_add = int(percentage * scaling_factor)
    ongoing_stages_list.extend([stage] * num_to_add)

# Ensure the list size is exactly num_ongoing_leads_to_generate by adding/removing randomly if needed
while len(ongoing_stages_list) < num_ongoing_leads_to_generate:
    # Add a random stage from the distribution
    stage_to_add = random.choices(list(ongoing_stage_distribution.keys()), weights=list(ongoing_stage_distribution.values()), k=1)[0]
    ongoing_stages_list.append(stage_to_add)
while len(ongoing_stages_list) > num_ongoing_leads_to_generate:
    # Remove a random stage
    ongoing_stages_list.pop(random.randint(0, len(ongoing_stages_list) - 1))


# Shuffle the list to ensure random selection
random.shuffle(ongoing_stages_list)
ongoing_stage_index = 0 # To cycle through the generated list

# Define the new, specific source categories
specific_sources = {
    "Inbound": [
        "Contact Us"
        "Demo Request",
       "Pricing Page",
        "Google",
        "Content Marketing",
        "Webinars",
        "LinkedIn",
        "Twitter",
        "Facebook",
        "Instagram posts",
         # Adding Customer-Driven Sources under Inbound for simplicity in this generation logic
        "Referrals from Existing Customers",
        "Word of Mouth"
    ],
    "Outbound": [
        "Cold Email Campaigns",
        "Cold Calling",
        "Telemarketing",
        "Google Ads",
        "LinkedIn Ads", "Facebook Ads", "Instagram Ads", "Twitter Ads",
        "Display Ads"," Retargeting Campaigns",
        "Third-Party Lead Generation Vendors", "Lists"
    ]
}

# Flatten the specific sources for easier random selection within a broader category
all_specific_inbound_sources = specific_sources["Inbound"]
all_specific_outbound_sources = specific_sources["Outbound"]


# Add predefined leads
# Ensure we don't exceed the total_leads_to_generate with predefined leads
predefined_leads_to_use = predefined_leads[:min(num_predefined_leads, total_leads_to_generate)]

for company_name, country, region, lead_id in predefined_leads_to_use:
    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}"

    created_date, last_updated = generate_timestamp()
    # Determine the broad deal source (Inbound or Outbound) first
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.4, 0.6])[0] # Adjusted weights for more Outbound

    # Assign a specific source based on the broad deal source
    if deal_source == "Inbound":
        source = random.choice(all_specific_inbound_sources)
    else: # Outbound
        source = random.choice(all_specific_outbound_sources)


    phone = generate_phone(country, region_mapping)

    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
    else:
        assigned_rep_id = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region and region_mapping.get(current_region, {}).get('services') else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region and region_mapping.get(current_region, {}).get('industries') else random.choice(all_industries)

    # Assign other random data first, as these influence the final stage determination
    engagement_level = random.choice(["High", "Medium", "Low"])
    follow_up_threshold = random.choice([3, 5, 7, 10])
    project_term = random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"])
    budget_tier = random.choice(["20-50L", "50-100L", "100L+"])
    priority = random.choice(["High", "Medium", "Low"])

    # Determine the final stage based on sequential progression and 'Deal Lost' conditions
    # For predefined leads, let's keep them mostly as Deal Won/Lost for consistency with initial data
    # A more sophisticated approach for ongoing leads will be applied to the additional leads
    if random.random() < 0.80: # Approximately 80% of predefined leads go to Won/Lost
        # Simulate progression to Won/Lost
        final_stage = "Lead Captured" # Start at the first stage
        for stage in funnel_stages:
            if stage == "Lead Captured":
                pass
            elif stage in potential_early_loss_stages:
                 met_early_loss_criteria = False
                 if stage in early_loss_criteria:
                     criteria = early_loss_criteria[stage]
                     budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
                     engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
                     priority_match = priority in criteria.get("priority", [priority])
                     random_chance_met = random.random() < criteria.get("random_chance", 0)

                     if budget_match and engagement_match and priority_match and random_chance_met:
                         met_early_loss_criteria = True

                 if met_early_loss_criteria:
                     final_stage = "Deal Lost"
                     break
                 else:
                     final_stage = stage
            elif stage == "Negotiation":
                 final_stage = stage
            if final_stage == "Negotiation":
                 met_negotiation_loss_criteria = False
                 criteria = negotiation_loss_criteria
                 budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
                 engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
                 random_chance_met = random.random() < criteria.get("random_chance", 0)

                 if budget_match and engagement_match and random_chance_met:
                     final_stage = "Deal Lost"
                 else:
                     final_stage = "Deal Won"
                 break
            else:
                final_stage = stage
    else:
        # For the remaining ~20% of predefined leads, assign a random intermediate stage from the ongoing list
        # Cycle through the ongoing_stages_list
        if ongoing_stages_list:
             final_stage = ongoing_stages_list[ongoing_stage_index % len(ongoing_stages_list)]
             ongoing_stage_index += 1
             final_stage = f"Ongoing - {final_stage}" # Rename to 'Ongoing'
        else:
             # Fallback if ongoing_stages_list is empty (should not happen with the list population logic)
             final_stage = random.choice(funnel_stages)
             if final_stage not in ["Deal Won", "Deal Lost"]:
                 final_stage = f"Ongoing - {final_stage}"
             else:
                 final_stage = "Lead Captured" # Default to first stage if somehow a final stage is picked


    leads_data.append({
        "Lead_ID": lead_id,
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": source, # Use the specific source
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": final_stage, # Use the determined final stage
        "Email": lead_email,
        "Phone": phone,
        "Priority": priority, # Use the generated priority
        "Last_Updated": last_updated,
        "Deal_Source": deal_source, # Keep the broad deal source
        "Company_ID": company_id,
        "Engagement_Level": engagement_level, # Use the generated engagement level
        "Follow_Up_Threshold": follow_up_threshold, # Use the generated follow up threshold
        "Project_Term": project_term, # Use the generated project term
        "Budget_Tier": budget_tier, # Use the generated budget tier
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })

# Calculate the number of additional leads needed after accounting for predefined leads used
num_additional_leads = total_leads_to_generate - len(predefined_leads_to_use)

# Get lists of all possible values for random generation
all_countries = [country for region_data in region_mapping.values() for country in data["countries"]]
all_regions = list(region_mapping.keys())

# Generate additional leads
for i in range(num_additional_leads):
    company_name = generate_random_company_name(company_name_parts["adjectives"], company_name_parts["nouns"])
    country = random.choice(all_countries)
    region = next(r for r, data in region_mapping.items() if country in data["countries"])

    company_abbreviation = get_abbreviated_company_name(company_name)
    # Adjust lead ID numbering to be sequential across all generated leads
    lead_id = f"LD{company_abbreviation}-{region}-{len(predefined_leads_to_use) + i + 1:04}"

    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}_{len(predefined_leads_to_use) + i + 1:04}"

    created_date, last_updated = generate_timestamp()
    # Determine the broad deal source (Inbound or Outbound) first
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.4, 0.6])[0] # Adjusted weights for more Outbound

    # Assign a specific source based on the broad deal source
    if deal_source == "Inbound":
        source = random.choice(all_specific_inbound_sources)
    else: # Outbound
        source = random.choice(all_specific_outbound_sources)

    phone = generate_phone(country, region_mapping)

    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
    else:
        assigned_rep_id = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region and region_mapping.get(current_region, {}).get('services') else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region and region_mapping.get(current_region, {}).get('industries') else random.choice(all_industries)


    # Assign other random data first, as these influence the final stage determination
    engagement_level = random.choice(["High", "Medium", "Low"])
    follow_up_threshold = random.choice([3, 5, 7, 10])
    project_term = random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"])
    budget_tier = random.choice(["20-50L", "50-100L", "100L+"])
    priority = random.choice(["High", "Medium", "Low"])

    # Determine the final stage: Approximately 30% ongoing, 70% Won/Lost
    # Use the ongoing_stages_list to assign stages for the ongoing leads
    if len(leads_data) < num_ongoing_leads_to_generate:
        # Assign an intermediate stage from the ongoing list
        if ongoing_stages_list:
             final_stage = ongoing_stages_list[ongoing_stage_index % len(ongoing_stages_list)]
             ongoing_stage_index += 1
             final_stage = f"Ongoing - {final_stage}" # Rename to 'Ongoing'
        else:
             # Fallback if ongoing_stages_list is empty (should not happen)
             final_stage = random.choice(funnel_stages)
             if final_stage not in ["Deal Won", "Deal Lost"]:
                 final_stage = f"Ongoing - {final_stage}"
             else:
                 final_stage = "Lead Captured" # Default to first stage

    else:
        # Simulate progression to Won/Lost for the remaining leads
        final_stage = "Lead Captured" # Start at the first stage
        for stage in funnel_stages:
            if stage == "Lead Captured":
                pass
            elif stage in potential_early_loss_stages:
                 met_early_loss_criteria = False
                 if stage in early_loss_criteria:
                     criteria = early_loss_criteria[stage]
                     budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
                     engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
                     priority_match = priority in criteria.get("priority", [priority])
                     random_chance_met = random.random() < criteria.get("random_chance", 0)

                     if budget_match and engagement_match and priority_match and random_chance_met:
                         met_early_loss_criteria = True

                 if met_early_loss_criteria:
                     final_stage = "Deal Lost"
                     break
                 else:
                     final_stage = stage

            elif stage == "Negotiation":
                 final_stage = stage

            if final_stage == "Negotiation":
                 met_negotiation_loss_criteria = False
                 criteria = negotiation_loss_criteria
                 budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
                 engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
                 random_chance_met = random.random() < criteria.get("random_chance", 0)

                 if budget_match and engagement_match and random_chance_met:
                     final_stage = "Deal Lost"
                 else:
                     final_stage = "Deal Won"
                 break
            else:
                final_stage = stage


    leads_data.append({
        "Lead_ID": lead_id,
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": source, # Use the specific source
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": final_stage, # Use the determined final stage
        "Email": lead_email,
        "Phone": phone,
        "Priority": priority, # Use the generated priority
        "Last_Updated": last_updated,
        "Deal_Source": deal_source, # Keep the broad deal source
        "Company_ID": company_id,
        "Engagement_Level": engagement_level, # Use the generated engagement level
        "Follow_Up_Threshold": follow_up_threshold, # Use the generated follow up threshold
        "Project_Term": project_term, # Use the generated project term
        "Budget_Tier": budget_tier, # Use the generated budget tier
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })


df_lead = pd.DataFrame(leads_data)

df_lead = df_lead[[
    "Lead_ID", "Lead_Name", "Company_Name", "Source", "Region_ID", "Rep_ID",
    "Created_Date", "Current_Stage", "Email", "Phone", "Priority", "Last_Updated",
    "Deal_Source", "Company_ID", "Engagement_Level", "Follow_Up_Threshold",
    "Project_Term", "Budget_Tier", "Country", "Company_Industry", "Service_Category"
]]


# After generating the initial df_lead, update the 'Current_Stage'
# to reflect the latest stage from the generated df_stage_history.
# This part needs to be executed after df_stage_history is generated in the next cell.
# This logic will be moved to a subsequent cell or integrated carefully if possible.


# ------------------------------------------
# Add Deal_Status_Category to df_lead
def categorize_deal_status(row, df_stage_history):
    if row['Current_Stage'] == 'Deal Won':
        return 'Deal Won'
    elif row['Current_Stage'] == 'Deal Lost':
        return 'Deal Lost'
    elif row['Current_Stage'].startswith('Ongoing - '):
        lead_id = row['Lead_ID']
        # Find the timestamp of the latest stage for this lead in stage history
        # We need the timestamp of the latest *ongoing* stage change to calculate duration
        latest_ongoing_stage_timestamp = df_stage_history[
            (df_stage_history['Lead_ID'] == lead_id) &
            (df_stage_history['Stage'].isin(funnel_stages)) # Consider only funnel stages for ongoing duration
        ]['Timestamp'].max()

        if pd.isna(latest_ongoing_stage_timestamp):
            # Fallback to Created_Date if no ongoing stage history found (should not happen with current logic)
            created_date_str = row['Created_Date']
            latest_ongoing_stage_timestamp = datetime.strptime(created_date_str, "%Y-%m-%d %H:%M:%S")


        # Calculate the duration since the last ongoing stage update
        duration_since_last_update = datetime.now() - latest_ongoing_stage_timestamp

        # Use a 2-month threshold for cold deals
        two_months_in_days = 60 # Approximation for 2 months

        if duration_since_last_update.days > two_months_in_days:
            return 'Ongoing - Cold'
        else:
            return 'Ongoing - Active'
    else:
        # For any other unexpected values in Current_Stage
        return 'Other' # Or map to 'Deal Lost' or 'Ongoing - Cold' as a default if preferred


# Apply the categorization function to create the new column
# This will initially be based on the generated df_stage_history.
# It will be more accurate after the stage_history cell is run and df_stage_history is populated.
# For now, it will use the potentially empty or incomplete df_stage_history from the last run.
# The final correct calculation will happen after the stage_history cell is executed.
# To ensure this runs correctly, the ETL cell (xX0Um6wCZejm) explicitly re-calculates this
# after loading the data from the Stage_History sheet.

# The current approach of calculating here before stage history is fully generated is problematic.
# We should either generate stage history first, or recalculate this column after.
# Given the current structure, the safest approach is to recalculate after stage_history is generated.
# However, to make the df_lead display immediately after this cell run reflect the intended logic,
# we will apply it here, but acknowledge it might be based on stale stage history data
# until the stage history cell and the ETL cell are run.

# Let's apply the categorization here, but with a note that it relies on df_stage_history
# which is fully populated in a subsequent cell.

# Temporarily handle the case where df_stage_history might not be fully populated yet
# (e.g., if this cell is run before the stage history generation cell)
# In a real pipeline, you'd ensure dependencies are met. Here, we'll just use an empty df
# as a fallback, which will result in all ongoing leads being categorized based on Created_Date
# until the stage history is available.
try:
    # Use the existing df_stage_history if it exists
    df_stage_history_for_categorization = df_stage_history
except NameError:
    # If df_stage_history is not defined, create an empty DataFrame with the expected columns
    print("Warning: df_stage_history not found. Categorizing ongoing deals based on Created_Date only.")
    df_stage_history_for_categorization = pd.DataFrame(columns=['Lead_ID', 'Stage', 'Timestamp'])


df_lead['Deal_Status_Category'] = df_lead.apply(
    lambda row: categorize_deal_status(row, df_stage_history_for_categorization),
    axis=1
)
# ------------------------------------------


# ------------------------------------------
# 5. Export or View
# ------------------------------------------
# Save to CSV
# df_lead.to_csv("crm_leads_generated.csv", index=False)

# Display sample
display(df_lead.head())
display(df_lead.tail())
print(f"Generated {len(df_lead)} leads.")

# Display the updated df_lead with the new column
display(df_lead['Deal_Status_Category'].value_counts())
display(df_lead['Source'].value_counts()) # Display the counts for the new 'Source' column

,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category,Deal_Status_Category
0,LDUBR-NA-001,Uber Lead,Uber,Facebook Ads,NA,BJ_NA,2023-12-23 15:15:30,Deal Won,contact_44@uber.com,+1-6973083715,...,Outbound,CMP_U,High,10,6-12 months,20-50L,USA,Technology,Data,Deal Won
1,LDWMT-NA-002,Walmart Lead,Walmart,Twitter Ads,NA,BJ_NA,2025-06-06 18:17:13,Deal Won,contact_76@walmart.com,+1-4721963142,...,Outbound,CMP_W,High,5,>24 months,100L+,USA,Finance,Cloud,Deal Won
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Retargeting Campaigns,NA,BJ_NA,2024-10-05 01:16:42,Deal Won,contact_4@fordmotorco.com,+1-9785819087,...,Outbound,CMP_FMC,High,7,12-24 months,100L+,USA,Technology,Cloud,Deal Won
3,LDAMZ-NA-004,Amazon Lead,Amazon,Contact UsDemo Request,NA,CL_NA,2024-10-07 03:06:21,Ongoing - Demo Scheduled,contact_82@amazon.com,+1-6636339804,...,Inbound,CMP_A,Medium,3,6-12 months,100L+,USA,Technology,Marketing,Ongoing - Cold
4,LDMST-NA-005,Microsoft Lead,Microsoft,Cold Calling,NA,AS_NA,2025-05-13 20:44:57,Deal Won,contact_44@microsoft.com,+1-7936939527,...,Outbound,CMP_M,Low,3,6-12 months,50-100L,USA,Finance,Marketing,Deal Won


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category,Deal_Status_Category
1995,LDGLOB-NA-1996,GlobalAnalytics Lead,GlobalAnalytics,Twitter,NA,BJ_NA,2025-03-08 07:18:37,Deal Won,contact_17@globalanalytics.com,+1-6480299406,...,Inbound,CMP_G_1996,Medium,3,0-6 months,50-100L,Canada,Healthcare,Cloud,Deal Won
1996,LDCREA-NA-1997,CreativeAnalytics Lead,CreativeAnalytics,Google Ads,NA,AS_NA,2023-10-07 18:01:43,Deal Lost,contact_88@creativeanalytics.com,+1-5405408767,...,Outbound,CMP_C_1997,Low,5,6-12 months,20-50L,Canada,Retail,Data,Deal Lost
1997,LDSTRA-NA-1998,StrategicTech Lead,StrategicTech,Content Marketing,NA,CL_NA,2023-10-10 06:55:48,Deal Won,contact_87@strategictech.com,+1-3136432860,...,Inbound,CMP_S_1998,Low,5,12-24 months,100L+,Canada,Retail,Data,Deal Won
1998,LDDYNA-NA-1999,DynamicAnalytics Lead,DynamicAnalytics,Cold Calling,NA,AS_NA,2024-09-28 19:08:48,Deal Won,contact_74@dynamicanalytics.com,+1-1497551417,...,Outbound,CMP_D_1999,High,3,0-6 months,50-100L,Canada,Healthcare,Marketing,Deal Won
1999,LDINNO-NA-2000,InnovativeVentures Lead,InnovativeVentures,Cold Calling,NA,AS_NA,2025-01-05 08:10:36,Deal Won,contact_29@innovativeventures.com,+1-1281785615,...,Outbound,CMP_I_2000,High,5,>24 months,20-50L,USA,Finance,Cloud,Deal Won


Generated 2000 leads.


,count
Deal_Status_Category,
Deal Won,1110
Ongoing - Cold,477
Deal Lost,374
Ongoing - Active,39


,count
Source,
Telemarketing,120
Third-Party Lead Generation Vendors,109
Retargeting Campaigns,108
Display Ads,106
Cold Calling,105
LinkedIn Ads,101
Facebook Ads,101
Cold Email Campaigns,101
Twitter Ads,100


In [10]:
#COMPANIES
import pandas as pd
import numpy as np
import random

industries = ["Technology", "Healthcare", "Finance", "Manufacturing", "Retail", "E-Commerce", "Green Tech", "Mobility", "Real Estate", "Banking", "Logistics", "Housing", "Energy"] # Expanded industries
sizes = ["Start-up", "Small business", "Medium Enterprise", "Large Enterprise"]
ownerships = ["Public Sector", "Private Sector", "Joint Venture", "Franchise"]
funding = ["Bootstrapped", "Seed", "Series A", "Series B", "IPO"]
interest_areas = ["Data", "Marketing", "Cloud", "Design", "HR", "AI", "Consulting", "Engineering", "Operations", "Staffing", "T&M"] # Expanded interest areas

company_data = []
added_company_ids = set()

# Add companies from df_lead
for index, row in df_lead.iterrows():
    company_id = row['Company_ID']
    company_name = row['Company_Name']
    country = row['Country']
    # Add company data only if not already added
    if company_id not in added_company_ids:
        # Attempt to get industry and service from region_mapping if available, otherwise use random
        # Find the country info in the region_mapping dictionary
        current_region_data = next((data for region, data in region_mapping.items() if country in data["countries"]), None)

        company_industry = random.choice(current_region_data.get('industries', industries)) if current_region_data and current_region_data.get('industries') else random.choice(industries)
        interest_area = random.choice(current_region_data.get('services', interest_areas)) if current_region_data and current_region_data.get('services') else random.choice(interest_areas)

        company_data.append({
            "Company_ID": company_id,
            "Company_Name": company_name,
            "Company_Industry": company_industry, # Use industry from region_mapping or random
            "Company_Size": random.choice(sizes),
            "Company_Ownership": random.choice(ownerships),
            "Company_Funding": random.choice(funding),
            "Country": country, # Use country from df_lead
            "Interest_Area": interest_area # Use service from region_mapping or random
        })
        added_company_ids.add(company_id)

# Generate additional companies if needed (optional, based on whether df_lead has all companies)
# If df_lead contains all the companies you need for df_company, you might not need this part.
# Assuming df_lead has the comprehensive list of companies you want in df_company.
# (No additional generation needed as df_lead now contains 10000 companies)

df_company = pd.DataFrame(company_data)
display(df_company.head())
display(df_company.tail())
print(f"Generated {len(df_company)} companies.")

,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
0,CMP_U,Uber,Finance,Medium Enterprise,Public Sector,Bootstrapped,USA,Marketing
1,CMP_W,Walmart,Retail,Large Enterprise,Joint Venture,IPO,USA,Cloud
2,CMP_FMC,Ford Motor Co,Healthcare,Large Enterprise,Joint Venture,IPO,USA,Data
3,CMP_A,Amazon,Technology,Medium Enterprise,Private Sector,Seed,USA,Marketing
4,CMP_M,Microsoft,Healthcare,Start-up,Private Sector,Series B,USA,Cloud


,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
1958,CMP_G_1996,GlobalAnalytics,Technology,Large Enterprise,Private Sector,IPO,Canada,Data
1959,CMP_C_1997,CreativeAnalytics,Retail,Large Enterprise,Joint Venture,IPO,Canada,Data
1960,CMP_S_1998,StrategicTech,Retail,Start-up,Public Sector,Series A,Canada,Marketing
1961,CMP_D_1999,DynamicAnalytics,Retail,Start-up,Franchise,Series A,Canada,Marketing
1962,CMP_I_2000,InnovativeVentures,Healthcare,Medium Enterprise,Public Sector,Seed,USA,Marketing


Generated 1963 companies.


In [11]:
#REGION
import pandas as pd
import numpy as np
import random

# Use unique Region_IDs from df_lead
region_ids = df_lead['Region_ID'].unique().tolist()
timezones = ["IST", "EST", "PST", "CET", "GMT"]
countries_in_leads = df_lead['Country'].unique().tolist() # Get countries from df_lead

region_data = []

# Define value mappings for company size for target calculation
company_size_value = {
    "Start-up": 1,
    "Small business": 5,
    "Medium Enterprise": 10,
    "Large Enterprise": 20
}

for region_id in region_ids:
    # Select a country that belongs to this region based on region_mapping
    country = random.choice([c for r, data in region_mapping.items() if r == region_id for c in data['countries'] if c in countries_in_leads])

    # Find reps assigned to this region
    reps_in_region = rep_df[rep_df['region'] == region_id]['rep_name'].tolist()
    region_head = random.choice(reps_in_region) if reps_in_region else "Unassigned"

    # Calculate Region_Target based on companies in the region
    companies_in_region = df_company[df_company['Country'].apply(lambda x: x in region_mapping.get(region_id, {}).get('countries', []))]

    # Calculate a score for each company based on size, ownership, funding, and interest area (simplified)
    # You can adjust weights and factors based on your desired logic
    company_scores = companies_in_region.apply(
        lambda row: company_size_value.get(row['Company_Size'], 0) *
                    (2 if row['Company_Ownership'] == 'Private Sector' else 1) *
                    (2 if row['Company_Funding'] in ['Series B', 'IPO'] else 1) *
                    (2 if row['Interest_Area'] in region_mapping.get(region_id, {}).get('services', []) else 1),
        axis=1
    )

    # Aggregate scores to get a raw region target
    raw_region_target = company_scores.sum()

    # Scale the raw target to a reasonable range (adjust scaling factor as needed)
    # Example: Scale by 100 to get targets in the thousands/millions
    region_target = round(raw_region_target * 500, 2) # Adjusted scaling factor

    region_data.append({
        "Region_ID": region_id,
        "Region_Name": f"Region_{region_id}",
        "Region_Head": region_head,
        "Timezone": random.choice(timezones),
        "Region_Target": region_target
    })

df_region = pd.DataFrame(region_data)
display(df_region.head())
print(f"Generated {len(df_region)} regions.")

,Region_ID,Region_Name,Region_Head,Timezone,Region_Target
0,NA,Region_NA,Bob Johnson,IST,30523000
1,EU,Region_EU,Fiona Green,EST,303000
2,APAC,Region_APAC,Kevin Martin,PST,388000
3,ME,Region_ME,Nina Adams,PST,40000
4,LA,Region_LA,Paula Young,PST,125000


Generated 6 regions.


In [12]:
#SALES_REP
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Use unique Rep_IDs and their corresponding regions from df_lead
sales_rep_data = []
added_rep_ids = set()

for index, row in df_lead.iterrows():
    rep_id = row['Rep_ID']
    region_id = row['Region_ID']
    # Add rep data only if not already added
    if rep_id not in added_rep_ids:
        # Find the rep name from the rep_df based on Rep_ID
        rep_info = rep_df[rep_df['rep_id'] == rep_id]
        rep_name = rep_info['rep_name'].iloc[0] if not rep_info.empty else f"Rep_{rep_id}"


        sales_rep_data.append({
            "Rep_ID": rep_id,
            "Rep_Name": rep_name, # Use name from rep_df
            "Email": f"{rep_id.lower()}@crmcorp.com", # Generate email based on Rep_ID
            "Phone": f"+91-9{random.randint(100000000,999999999)}", # Keep existing phone generation for now
            "Region_ID": region_id,
            "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
            "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
            "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
            "Total_Deals_Closed": random.randint(0, 500),
            "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
            "Win_rate": round(random.uniform(0.1, 0.9), 2)
        })
        added_rep_ids.add(rep_id)

# Add any reps from rep_df who are not assigned to any lead in df_lead
reps_not_in_leads = rep_df[~rep_df['rep_id'].isin(added_rep_ids)]
for index, rep_row in reps_not_in_leads.iterrows():
    rep_id = rep_row['rep_id']
    region_id = rep_row['region']
    rep_name = rep_row['rep_name']

    sales_rep_data.append({
        "Rep_ID": rep_id,
        "Rep_Name": rep_name,
        "Email": f"{rep_id.lower()}@crmcorp.com",
        "Phone": f"+91-9{random.randint(100000000,999999999)}",
        "Region_ID": region_id,
        "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
        "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
        "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
        "Total_Deals_Closed": random.randint(0, 500),
        "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
        "Win_rate": round(random.uniform(0.1, 0.9), 2)
    })


df_sales_rep = pd.DataFrame(sales_rep_data)
display(df_sales_rep.head())
display(df_sales_rep.tail())
print(f"Generated {len(df_sales_rep)} sales reps.")

,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
0,BJ_NA,Bob Johnson,bj_na@crmcorp.com,+91-9500731582,NA,2020-07-10,On Leave,2.6,166,4.58,0.14
1,CL_NA,Charlie Lee,cl_na@crmcorp.com,+91-9595556646,NA,2020-04-20,Active,3.7,241,8.46,0.59
2,AS_NA,Alice Smith,as_na@crmcorp.com,+91-9864762441,NA,2020-09-07,Active,3.2,428,17.09,0.67
3,DK_NA,Diana King,dk_na@crmcorp.com,+91-9536679381,NA,2020-07-15,On Leave,1.2,344,23.25,0.34
4,HB_EU,Hannah Black,hb_eu@crmcorp.com,+91-9185527756,EU,2022-01-27,Active,4.4,432,26.04,0.58


,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
15,RA_LA,Rachel Allen,ra_la@crmcorp.com,+91-9989176162,LA,2020-10-16,Inactive,1.5,310,19.96,0.31
16,PY_LA,Paula Young,py_la@crmcorp.com,+91-9235392863,LA,2022-07-23,Active,1.2,497,14.02,0.15
17,SW_AF,Steve Wright,sw_af@crmcorp.com,+91-9830973668,AF,2021-06-01,On Leave,3.6,305,4.14,0.22
18,TB_AF,Tina Baker,tb_af@crmcorp.com,+91-9645891291,AF,2021-12-10,Active,4.9,468,5.48,0.85
19,QH_LA,Quinn Hall,qh_la@crmcorp.com,+91-9762463105,LA,2021-05-27,Inactive,4.2,13,24.03,0.86


Generated 20 sales reps.


In [13]:
#STAGE HISTORY
import pandas as pd
import random
from datetime import datetime, timedelta

# Define stages in the desired funnel order based on user provided ranges
funnel_stages = [
    "Lead Captured",
    "Qualified",
    "Contacted",
    "Responded",
    "Demo Scheduled",
    "Demo",
    "Demo Completed",
    "Proposal Sent",
    "Proposal",
    "Negotiation"
]

stage_history_data = []
history_id = 1

# Define realistic time ranges for progression between stages (in days)
# Using the specific ranges provided by the user
stage_time_ranges = {
    "Lead Captured": (2, 7),        # 2–7 days to qualification
    "Qualified": (4, 9),            # 4–9 days to contact
    "Contacted": (5, 12),           # 5–12 days to responded/demo
    "Responded": (3, 6),            # 3–6 days to demo scheduled
    "Demo Scheduled": (4, 8),       # 4–8 days to demo
    "Demo": (2, 3),                 # 2–3 days to complete demo
    "Demo Completed": (4, 9),       # 4–9 days to proposal
    "Proposal Sent": (5, 11),       # 5–11 days to proposal
    "Proposal": (6, 13),            # 6–13 days to negotiation
    "Negotiation": (7, 15)          # 7–15 days to close
}

def add_random_time_of_day(date):
    """Adds a random time of day (hour, minute, second) to a date."""
    # Ensure 'date' is a datetime object before adding timedelta
    if isinstance(date, datetime):
        return date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    elif isinstance(date, pd.Timestamp):
         return date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    else:
        # If it's a date object, convert to datetime before adding time
        return datetime(date.year, date.month, date.day) + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))


for index, lead in df_lead.iterrows():
    lead_id = lead.Lead_ID
    final_stage = lead['Current_Stage']

    # Determine the sequence of stages the lead went through, stopping at the Current_Stage
    # Handle 'Ongoing' stages by extracting the base stage name
    base_final_stage = final_stage.replace("Ongoing - ", "")

    try:
        if base_final_stage in ["Deal Won", "Deal Lost"]:
             # Find the index of the final stage in the funnel_stages.
             # If the final stage is not in funnel_stages (i.e., Won/Lost),
             # the history goes up to Negotiation (the last stage before Won/Lost).
             try:
                 final_stage_index_in_funnel = funnel_stages.index(base_final_stage)
                 # If the final stage is in funnel_stages, history goes up to that stage.
                 ordered_stages = funnel_stages[:final_stage_index_in_funnel + 1]
             except ValueError:
                 # If final stage is 'Deal Won' or 'Deal Lost', history goes up to 'Negotiation'
                 if "Negotiation" in funnel_stages:
                     final_history_stage_index = funnel_stages.index("Negotiation")
                     ordered_stages_for_history = funnel_stages[:final_history_stage_index + 1]
                 else:
                     # Fallback if Negotiation is not in funnel_stages
                     ordered_stages_for_history = funnel_stages # Include all stages if structure is different

                 # Add the actual final stage ('Deal Won' or 'Deal Lost') as the last entry in history
                 ordered_stages = ordered_stages_for_history + [base_final_stage]

        elif base_final_stage in funnel_stages:
            # If the final stage is an intermediate stage, history goes up to that stage
            final_stage_index = funnel_stages.index(base_final_stage)
            ordered_stages = funnel_stages[:final_stage_index + 1]
        else:
             # If the assigned stage is not in funnel_stages or is invalid, default to "Lead Captured"
             print(f"Warning: Lead {lead.Lead_ID} has an invalid final stage '{final_stage}'. Setting history to 'Lead Captured'.")
             ordered_stages = ["Lead Captured"]
             base_final_stage = "Lead Captured" # Update base_final_stage variable as well


    except ValueError:
        # If the assigned stage is not in funnel_stages or is invalid, default to "Lead Captured"
        print(f"Warning: Lead {lead.Lead_ID} has an invalid final stage '{final_stage}'. Setting history to 'Lead Captured'.")
        ordered_stages = ["Lead Captured"]
        base_final_stage = "Lead Captured" # Update base_final_stage variable as well


    start_date_str = lead.Created_Date
    last_updated_date_str = lead.Last_Updated

    # Convert to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S")
    last_updated_date = datetime.strptime(last_updated_date_str, "%Y-%m-%d %H:%M:%S")

    total_duration_seconds = (last_updated_date - start_date).total_seconds()
    num_transitions = len(ordered_stages) - 1

    # Generate timestamps for each stage
    current_timestamp = start_date
    for seq, stage in enumerate(ordered_stages, start=1):
        timestamp = current_timestamp

        # If it's the first stage, use the created date with random time
        if seq == 1:
            timestamp = add_random_time_of_day(start_date)
             # Ensure the generated timestamp is not before the original created_date if original had time
            if timestamp < start_date:
                 timestamp = start_date + timedelta(seconds=random.randint(1, 60)) # Ensure strictly after if on same day


        # If it's the last stage, use the last updated date
        elif seq == len(ordered_stages):
            timestamp = last_updated_date
            # Ensure the last stage timestamp is not before the previous stage timestamp
            if timestamp <= current_timestamp:
                 timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60)) # Add a small increment if needed


        # For intermediate stages, distribute the remaining time
        else:
             # Calculate remaining time and number of remaining transitions
             remaining_duration_seconds = (last_updated_date - current_timestamp).total_seconds()
             remaining_transitions = len(ordered_stages) - seq

             if remaining_transitions > 0:
                 # Determine the time to add for the current transition (from 'stage' to 'next_stage')
                 # Use a random value within a range around the estimated average, but bounded by stage time ranges
                 # The time range for the current transition is based on the *current* stage (which is stage)
                 min_days, max_days = stage_time_ranges.get(stage, (1, 1))
                 min_seconds = min_days * 24 * 3600
                 max_seconds = max_days * 24 * 3600

                 # Calculate a random increment, ensuring it's positive and within bounds relative to remaining time
                 # Avoid adding more time than is available until the last updated date
                 # Ensure there is enough time left for the minimum duration of the *remaining* stages (excluding the current transition)
                 # This logic is tricky because we don't know the exact future stages for ongoing leads.
                 # For simplicity and to ensure progression, we'll distribute the time evenly
                 # based on the remaining stages *in the potential path* (up to Negotiation or the assigned Current_Stage).

                 # Determine the stages remaining in the history path (excluding the current stage and the final history stage)
                 remaining_stages_in_path = ordered_stages[seq:len(ordered_stages)-1]

                 estimated_min_remaining_days = sum(stage_time_ranges.get(s, (1, 1))[0] for s in remaining_stages_in_path)

                 min_seconds_for_remaining_stages = estimated_min_remaining_days * 24 * 3600
                 max_possible_increment_seconds = remaining_duration_seconds - min_seconds_for_remaining_stages # Ensure enough time for future min stages


                 # Ensure max_possible_increment_seconds is non-negative and at least 1 second if remaining_transitions > 0
                 max_possible_increment_seconds = max(1, max_possible_increment_seconds) if remaining_transitions > 0 else 0


                 # Select a random time to add, ensuring it is at least 1 second and does not exceed available time
                 # Also try to respect the stage_time_ranges for the current stage
                 # Ensure the lower bound is at least 1 second
                 lower_bound_seconds = max(min_seconds, 1)
                 upper_bound_seconds = min(max_seconds, max_possible_increment_seconds, remaining_duration_seconds)

                 # Ensure upper_bound_seconds is at least lower_bound_seconds
                 upper_bound_seconds = max(lower_bound_seconds, upper_bound_seconds)


                 time_to_add_seconds = random.uniform(lower_bound_seconds, upper_bound_seconds)


                 # Ensure the next timestamp is strictly after the current timestamp
                 next_timestamp = current_timestamp + timedelta(seconds=time_to_add_seconds)
                 while next_timestamp <= current_timestamp:
                      next_timestamp = current_timestamp + timedelta(seconds=random.uniform(1, 60)) # Add a small increment


                 timestamp = next_timestamp
             else:
                 # Should not happen for intermediate stages if logic is correct, but as a safeguard
                 timestamp = current_timestamp + timedelta(seconds=random.uniform(1, 60)) # Add a small increment


        stage_history_data.append({
            "History_ID": history_id,
            "Lead_ID": lead_id,
            "Stage": ordered_stages[seq-1], # Use the stage name from the ordered_stages list
            "Timestamp": timestamp,
            "Rep_ID": lead.Rep_ID,
            "Stage_Sequence": seq,
            "Notes": f"Notes for stage {ordered_stages[seq-1]} for Lead {lead_id}" # Use the stage name
        })
        history_id += 1
        current_timestamp = timestamp


df_stage_history = pd.DataFrame(stage_history_data)

df_stage_history['Timestamp'] = pd.to_datetime(df_stage_history['Timestamp'])
df_stage_history = df_stage_history.sort_values(by=['Lead_ID', 'Timestamp']).reset_index(drop=True)

# Recalculate Stage_Engagement (in Days) based on sorted timestamps per lead
stage_engagement_days = []

for lead_id, group in df_stage_history.groupby('Lead_ID'):
    group = group.sort_values(by='Timestamp').reset_index(drop=True)
    for i in range(len(group)):
        if i == 0:
            days = 0 # The first stage has 0 days engagement within the history
        else:
            time_diff = group.iloc[i]['Timestamp'] - group.iloc[i-1]['Timestamp']
            days = round(time_diff.total_seconds() / (24 * 3600))
            days = max(days, 0) # Ensure non-negative days
        stage_engagement_days.append(days)


df_stage_history['Stage_Engagement (in Days)'] = stage_engagement_days

# Update df_lead's Current_Stage based on the latest stage in df_stage_history
# This step is actually redundant now that df_lead's Current_Stage is correctly generated.
# We can remove this part to avoid potential inconsistencies if the history generation
# somehow doesn't perfectly align with the df_lead's Current_Stage.
# latest_stage_indices = df_stage_history.groupby('Lead_ID')['Timestamp'].idxmax()
# latest_stages = df_stage_history.loc[latest_stage_indices][['Lead_ID', 'Stage']]
# latest_stages.rename(columns={'Stage': 'Actual_Current_Stage'}, inplace=True)
# df_lead = df_lead.merge(latest_stages, on='Lead_ID', how='left', suffixes=('', '_latest_history'))
# df_lead['Current_Stage'] = df_lead['Actual_Current_Stage'].fillna(df_lead['Current_Stage'])
# df_lead = df_lead.drop(columns=['Actual_Current_Stage'])


display(df_lead.head())
display(df_lead.tail())
print(f"Generated {len(df_lead)} leads.")

display(df_stage_history.head())
display(df_stage_history.tail())
print(f"Generated {len(df_stage_history)} stage history entries.")

,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category,Deal_Status_Category
0,LDUBR-NA-001,Uber Lead,Uber,Facebook Ads,NA,BJ_NA,2023-12-23 15:15:30,Deal Won,contact_44@uber.com,+1-6973083715,...,Outbound,CMP_U,High,10,6-12 months,20-50L,USA,Technology,Data,Deal Won
1,LDWMT-NA-002,Walmart Lead,Walmart,Twitter Ads,NA,BJ_NA,2025-06-06 18:17:13,Deal Won,contact_76@walmart.com,+1-4721963142,...,Outbound,CMP_W,High,5,>24 months,100L+,USA,Finance,Cloud,Deal Won
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Retargeting Campaigns,NA,BJ_NA,2024-10-05 01:16:42,Deal Won,contact_4@fordmotorco.com,+1-9785819087,...,Outbound,CMP_FMC,High,7,12-24 months,100L+,USA,Technology,Cloud,Deal Won
3,LDAMZ-NA-004,Amazon Lead,Amazon,Contact UsDemo Request,NA,CL_NA,2024-10-07 03:06:21,Ongoing - Demo Scheduled,contact_82@amazon.com,+1-6636339804,...,Inbound,CMP_A,Medium,3,6-12 months,100L+,USA,Technology,Marketing,Ongoing - Cold
4,LDMST-NA-005,Microsoft Lead,Microsoft,Cold Calling,NA,AS_NA,2025-05-13 20:44:57,Deal Won,contact_44@microsoft.com,+1-7936939527,...,Outbound,CMP_M,Low,3,6-12 months,50-100L,USA,Finance,Marketing,Deal Won


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category,Deal_Status_Category
1995,LDGLOB-NA-1996,GlobalAnalytics Lead,GlobalAnalytics,Twitter,NA,BJ_NA,2025-03-08 07:18:37,Deal Won,contact_17@globalanalytics.com,+1-6480299406,...,Inbound,CMP_G_1996,Medium,3,0-6 months,50-100L,Canada,Healthcare,Cloud,Deal Won
1996,LDCREA-NA-1997,CreativeAnalytics Lead,CreativeAnalytics,Google Ads,NA,AS_NA,2023-10-07 18:01:43,Deal Lost,contact_88@creativeanalytics.com,+1-5405408767,...,Outbound,CMP_C_1997,Low,5,6-12 months,20-50L,Canada,Retail,Data,Deal Lost
1997,LDSTRA-NA-1998,StrategicTech Lead,StrategicTech,Content Marketing,NA,CL_NA,2023-10-10 06:55:48,Deal Won,contact_87@strategictech.com,+1-3136432860,...,Inbound,CMP_S_1998,Low,5,12-24 months,100L+,Canada,Retail,Data,Deal Won
1998,LDDYNA-NA-1999,DynamicAnalytics Lead,DynamicAnalytics,Cold Calling,NA,AS_NA,2024-09-28 19:08:48,Deal Won,contact_74@dynamicanalytics.com,+1-1497551417,...,Outbound,CMP_D_1999,High,3,0-6 months,50-100L,Canada,Healthcare,Marketing,Deal Won
1999,LDINNO-NA-2000,InnovativeVentures Lead,InnovativeVentures,Cold Calling,NA,AS_NA,2025-01-05 08:10:36,Deal Won,contact_29@innovativeventures.com,+1-1281785615,...,Outbound,CMP_I_2000,High,5,>24 months,20-50L,USA,Finance,Cloud,Deal Won


Generated 2000 leads.


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
0,321,LDABB-EU-032,Lead Captured,2025-06-14 20:08:52,FG_EU,1,Notes for stage Lead Captured for Lead LDABB-E...,0
1,322,LDABB-EU-032,Qualified,2025-06-18 20:08:52,FG_EU,2,Notes for stage Qualified for Lead LDABB-EU-032,4
2,323,LDABB-EU-032,Contacted,2025-06-23 20:08:52,FG_EU,3,Notes for stage Contacted for Lead LDABB-EU-032,5
3,324,LDABB-EU-032,Responded,2025-06-26 20:08:52,FG_EU,4,Notes for stage Responded for Lead LDABB-EU-032,3
4,325,LDABB-EU-032,Demo Scheduled,2025-06-30 20:08:52,FG_EU,5,Notes for stage Demo Scheduled for Lead LDABB-...,4


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
19190,857,LDZNR-APAC-087,Demo Completed,2024-12-13 06:59:39,LS_APAC,7,Notes for stage Demo Completed for Lead LDZNR-...,4
19191,858,LDZNR-APAC-087,Proposal Sent,2024-12-18 06:59:39,LS_APAC,8,Notes for stage Proposal Sent for Lead LDZNR-A...,5
19192,859,LDZNR-APAC-087,Proposal,2024-12-24 06:59:39,LS_APAC,9,Notes for stage Proposal for Lead LDZNR-APAC-087,6
19193,860,LDZNR-APAC-087,Negotiation,2024-12-31 06:59:39,LS_APAC,10,Notes for stage Negotiation for Lead LDZNR-APA...,7
19194,861,LDZNR-APAC-087,Deal Won,2024-12-31 07:00:04,LS_APAC,11,Notes for stage Deal Won for Lead LDZNR-APAC-087,0


Generated 19195 stage history entries.


In [14]:
# DEALS GENERATION
def generate_deals(df_lead, df_stage_history):
    deals_data = []
    deal_id_counter = 1

    # Define amount ranges for each budget tier (in Lakhs)
    # Assuming 'L' means Lakhs and '100L+' means 1 Crore or more
    budget_tier_amounts = {
        "20-50L": (20, 50),
        "50-100L": (50, 100),
        "100L+": (100, 500) # Assuming 1 Cr to 5 Cr for 100L+
    }

    # Include both closed and ongoing leads
    leads_to_process = df_lead[df_lead['Current_Stage'].isin(['Deal Won', 'Deal Lost']) | df_lead['Current_Stage'].str.startswith('Ongoing - ')].copy()

    for index, lead in leads_to_process.iterrows():
        lead_id = lead['Lead_ID']
        current_stage = lead['Current_Stage']
        deal_status_category = lead['Deal_Status_Category'] # Get the status category

        lead_captured_ts = df_stage_history[
            (df_stage_history['Lead_ID'] == lead_id) & (df_stage_history['Stage'] == 'Lead Captured')
        ]['Timestamp'].min()

        if pd.isna(lead_captured_ts):
            # Skip leads with no 'Lead Captured' stage history
            print(f"Warning: Skipping lead {lead_id} as it has no 'Lead Captured' stage history.")
            continue


        # Determine the amount based on the Budget_Tier (same logic for ongoing and closed)
        budget_tier = lead['Budget_Tier']
        amount_range = budget_tier_amounts.get(budget_tier, (1, 10)) # Default to small range if tier not found

        # Generate a random amount within the range, in Lakhs
        amount_in_lakhs = round(random.uniform(amount_range[0], amount_range[1]), 2)

        # Format the amount with 'Lakhs' or 'Crore' for display purposes (optional, but can be useful)
        if amount_in_lakhs >= 100:
             # Convert Lakhs to Crores for display
             amount_display = f"{amount_in_lakhs / 100:.2f} Crore"
             amount_numeric = amount_in_lakhs * 100000 # Convert Lakhs to Rupees for numeric value (assuming 1 Lakh = 100,000 INR)
        else:
             amount_display = f"{amount_in_lakhs:.2f} Lakhs"
             amount_numeric = amount_in_lakhs * 100000 # Convert Lakhs to Rupees for numeric value


        closed_date = None
        total_duration_days = None

        if current_stage in ['Deal Won', 'Deal Lost']:
            closed_ts = df_stage_history[
                (df_stage_history['Lead_ID'] == lead_id) & (df_stage_history['Stage'].isin(['Deal Won', 'Deal Lost']))
            ]['Timestamp'].max()
            if pd.notna(closed_ts):
                closed_date = closed_ts.date().strftime("%Y-%m-%d")
                total_duration_days = (closed_ts - lead_captured_ts).days
        elif current_stage.startswith('Ongoing - '):
            # For ongoing deals, calculate duration from Created_Date to current date
            total_duration_days = (datetime.now() - lead_captured_ts).days

        # Update the Status based on Deal_Status_Category
        status = deal_status_category if deal_status_category in ['Ongoing - Active', 'Ongoing - Cold'] else current_stage


        deals_data.append({
            "Deal_ID": f"DEAL_{deal_id_counter:04}",
            "Lead_ID": lead_id,
            "Company_ID": lead['Company_ID'],
            "Amount_Display": amount_display, # Keep the formatted amount as a display column
            "Amount_Numeric": amount_numeric, # Add the numeric amount column
            "Status": status, # Use the updated status
            "Created_Date": lead_captured_ts.date().strftime("%Y-%m-%d"),
            "Closed_Date": closed_date, # This will be None for ongoing deals
            "Rep_ID": lead['Rep_ID'],
            "Total_Duration_Days": total_duration_days
        })
        deal_id_counter += 1

    return pd.DataFrame(deals_data)

df_deals = generate_deals(df_lead, df_stage_history)
display(df_deals.head())
display(df_deals.tail())
print(f"Generated {len(df_deals)} deals.")

,Deal_ID,Lead_ID,Company_ID,Amount_Display,Amount_Numeric,Status,Created_Date,Closed_Date,Rep_ID,Total_Duration_Days
0,DEAL_0001,LDUBR-NA-001,CMP_U,25.78 Lakhs,2578000.0,Deal Won,2023-12-23,2024-02-01,BJ_NA,40
1,DEAL_0002,LDWMT-NA-002,CMP_W,4.03 Crore,40288000.0,Deal Won,2025-06-07,2025-07-17,BJ_NA,40
2,DEAL_0003,LDFRD-NA-003,CMP_FMC,3.82 Crore,38188000.0,Deal Won,2024-10-05,2024-11-21,BJ_NA,47
3,DEAL_0004,LDAMZ-NA-004,CMP_A,4.35 Crore,43471000.0,Ongoing - Cold,2024-10-07,None,CL_NA,318
4,DEAL_0005,LDMST-NA-005,CMP_M,77.41 Lakhs,7741000.0,Deal Won,2025-05-14,2025-07-09,AS_NA,56


,Deal_ID,Lead_ID,Company_ID,Amount_Display,Amount_Numeric,Status,Created_Date,Closed_Date,Rep_ID,Total_Duration_Days
1995,DEAL_1996,LDGLOB-NA-1996,CMP_G_1996,99.71 Lakhs,9971000.0,Deal Won,2025-03-08,2025-04-17,BJ_NA,40
1996,DEAL_1997,LDCREA-NA-1997,CMP_C_1997,35.00 Lakhs,3500000.0,Deal Lost,2023-10-08,2023-11-17,AS_NA,40
1997,DEAL_1998,LDSTRA-NA-1998,CMP_S_1998,2.61 Crore,26102000.0,Deal Won,2023-10-11,2023-12-01,CL_NA,51
1998,DEAL_1999,LDDYNA-NA-1999,CMP_D_1999,96.96 Lakhs,9696000.0,Deal Won,2024-09-29,2024-11-08,AS_NA,40
1999,DEAL_2000,LDINNO-NA-2000,CMP_I_2000,44.36 Lakhs,4436000.0,Deal Won,2025-01-06,2025-02-15,AS_NA,40


Generated 2000 deals.


In [15]:
!pip install --upgrade gspread gspread_dataframe

import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()


In [16]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 625.2 kB/s eta 0:00:00


In [17]:
# Use the spreadsheet name you created
spreadsheet = client.open("CRM_Capstone")

# Sheet mappings
sheet_mapping = {
    "Leads": df_lead,
    "Sales_Reps": df_sales_rep,
    "Regions": df_region,
    "Stage_History": df_stage_history,
    "Deals":df_deals,
    "Companies": df_company
}

# Write to each sheet
for sheet_name, df in sheet_mapping.items():
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=df.shape[0], cols=df.shape[1])

    worksheet.clear()
    set_with_dataframe(worksheet, df)

print("All DataFrames written to Google Sheets.")

All DataFrames written to Google Sheets.


In [18]:
# Full ETL pipeline (Google Sheet -> Clean -> Google Sheet -> SQLite + SQL Modeling + Google Drive CSV + Supabase + BigQuery)

import pandas as pd
import re
import sqlite3
import gspread
from gspread_dataframe import set_with_dataframe
import urllib.parse # Import for URL encoding
from sqlalchemy import create_engine # Import for SQLAlchemy engine
from google.colab import auth # Import auth for Google Colab authentication
from oauth2client.service_account import ServiceAccountCredentials # Import for service account credentials
from google.colab import drive # Import drive for mounting Google Drive
from google.cloud import bigquery # Import BigQuery client


# ---------- Google Sheets Authentication and Connection ----------
try:
    # Attempt to use the existing client object if defined
    # Renaming to google_sheets_client to avoid conflict with bigquery.Client
    google_sheets_client = client
except NameError:
    print("Google Sheets client not found. Attempting to authenticate and create a new client.")
    # Authenticate using service account credentials
    try:
        # Assuming the creds.json file path is defined or can be accessed
        # If your creds.json path is different, update this line
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
        google_sheets_client = gspread.authorize(creds)
        print("Successfully authenticated and created Google Sheets client.")
    except FileNotFoundError:
        print("Error: 'creds.json' not found. Please ensure your service account credentials file is in the correct path.")
        # Fallback to Colab authentication if service account fails
        try:
            auth.authenticate_user()
            google_sheets_client = gspread.service_account_manager().authorize()
            print("Successfully authenticated using Google Colab user credentials.")
        except Exception as e:
            print(f"Error authenticating with Google Colab credentials: {e}")
            raise # Re-raise if both authentication methods fails
    except Exception as e:
        print(f"An error occurred during Google Sheets authentication: {e}")
        raise # Re-raise other exceptions


# Assuming 'spreadsheet' object is available from a previous cell that connected to Google Sheets
# If not, try to open it here using the authenticated client
try:
    spreadsheet
except NameError:
    print("Spreadsheet object not found. Attempting to open spreadsheet.")
    try:
        spreadsheet = google_sheets_client.open("CRM_Capstone") # Replace with your spreadsheet name
        print("Successfully opened spreadsheet.")
    except Exception as e:
        print(f"An error occurred while opening the spreadsheet: {e}")
        raise # Re-raise the error if spreadsheet cannot be opened


# ---------- Helper functions (from cell 8AxwbCx9My_R) ----------
def strip_strings(df):
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].astype(str).str.strip().replace("nan", pd.NA)
    return df

def fix_email(email):
    if pd.isna(email):
        return email
    email = email.strip().lower()
    email = re.sub(r"[ ']", "", email)
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return email if re.match(pattern, email) else pd.NA

def clean_region_id(df):
    region_cols = [c for c in df.columns if "Region_ID" in c]
    for col in region_cols:
        df[col] = df[col].replace("", "NA").fillna("NA")
    return df

def convert_dates(df, date_cols, to_date_only=False):
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            if to_date_only:
                df[col] = df[col].dt.date
    return df

def clean_all_data(data):
    # Leads
    leads = strip_strings(data["Leads"].copy()) # Use .copy() to avoid SettingWithCopyWarning
    leads = clean_region_id(leads)
    leads.drop_duplicates(inplace=True)
    if "Email" in leads.columns:
        leads["Email"] = leads["Email"].apply(fix_email)
    leads = convert_dates(leads, [c for c in leads.columns if "Date" in c])

    # Sales_Reps
    sales_reps = strip_strings(data["Sales_Reps"].copy()) # Use .copy()
    sales_reps = clean_region_id(sales_reps)
    sales_reps.drop_duplicates(inplace=True)
    if "Email" in sales_reps.columns:
        sales_reps["Email"] = sales_reps["Email"].apply(fix_email)
    sales_reps = convert_dates(sales_reps, ["Join_Date"], to_date_only=True)

    # Regions
    regions = strip_strings(data["Regions"].copy()) # Use .copy()
    regions = clean_region_id(regions)
    regions.drop_duplicates(inplace=True)

    # Companies
    companies = strip_strings(data["Companies"].copy()) # Use .copy()
    companies = clean_region_id(companies)
    companies.drop_duplicates(inplace=True)

    # Stage_History
    stage_history = strip_strings(data["Stage_History"].copy()) # Use .copy()
    stage_history = clean_region_id(stage_history)
    stage_history.drop_duplicates(inplace=True)
    stage_history = convert_dates(stage_history, ["Timestamp"])

    # Deals
    deals = strip_strings(data["Deals"].copy()) # Use .copy()
    deals = clean_region_id(deals)
    deals.drop_duplicates(inplace=True)
    deals = convert_dates(deals, ["Created_Date", "Closed_Date"], to_date_only=True)


    data["Leads"] = leads
    data["Sales_Reps"] = sales_reps
    data["Regions"] = regions
    data["Companies"] = companies
    data["Stage_History"] = stage_history
    data["Deals"] = deals
    return data


# ---------- Step 1: Extract from Google Sheet ----------
# Use the spreadsheet object obtained from the previous successful connection
# Define sheets to process - ensure these match your Google Sheet tab names
sheets = ["Leads", "Sales_Reps", "Regions", "Companies", "Stage_History", "Deals"]
data_from_sheets = {}
for sheet_name in sheets:
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        # Get all records as a list of dictionaries, then convert to DataFrame
        data_from_sheets[sheet_name] = pd.DataFrame(worksheet.get_all_records())
        print(f"Successfully loaded data from sheet: {sheet_name}")
    except gspread.WorksheetNotFound:
        print(f"Warning: Sheet '{sheet_name}' not found in the spreadsheet. Skipping.")
    except Exception as e:
        print(f"An error occurred while loading data from sheet '{sheet_name}': {e}")


# ---------- Step 2: Transform (Clean Data) ----------
cleaned_data = clean_all_data(data_from_sheets)
print("✅ Data Cleaning Complete.")

# ---------- Step 3: Load cleaned data back to Google Sheet ----------
for sheet_name, df in cleaned_data.items():
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        # Clear existing data before writing
        worksheet.clear()
        # Write the cleaned dataframe back to the sheet
        set_with_dataframe(worksheet, df)
        print(f"Successfully wrote cleaned data back to sheet: {sheet_name}")
    except gspread.WorksheetNotFound:
        print(f"Warning: Sheet '{sheet_name}' not found. Cannot write data.")
    except Exception as e:
        print(f"An error occurred while writing data to sheet '{sheet_name}': {e}")
print("💾 Cleaned data written back to Google Sheet.")

# ---------- Step 4: Load cleaned data as CSV files to Google Drive ----------
drive_folder_path = "/content/drive/MyDrive/CAPSTONE_FINAL/Generated_Data" # Specify your desired folder path

# Create the folder if it doesn't exist (optional, but good practice)
import os
if not os.path.exists(drive_folder_path):
    os.makedirs(drive_folder_path)
    print(f"Created Google Drive folder: {drive_folder_path}")


for sheet_name, df in cleaned_data.items():
    output_csv_path = os.path.join(drive_folder_path, f"{sheet_name}_cleaned.csv")
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"Successfully saved cleaned data to CSV in Google Drive: {output_csv_path}")
    except Exception as e:
        print(f"An error occurred while saving cleaned data to CSV for {sheet_name}: {e}")
print("💾 Cleaned data saved as CSV files in Google Drive.")


# ---------- Step 5: Load into SQLite ----------
db_path = "/content/drive/MyDrive/CAPSTONE_FINAL/CRM_Capstone_DB.sqlite"
conn = sqlite3.connect(db_path)
for sheet_name, df in cleaned_data.items():
    # Ensure column names are valid SQLite identifiers (replace spaces/special chars)
    df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=True)
    # Print column names for verification, especially for Stage_History
    if sheet_name == "Stage_History":
        print(f"Columns in {sheet_name} before loading to SQLite: {df.columns.tolist()}")
    df.to_sql(sheet_name, conn, if_exists="replace", index=False)
print(f"✅ Data loaded into {db_path}")


# ---------- Step 6: Load cleaned data into BigQuery ----------
print("⬆️ Attempting to upload cleaned data to BigQuery...")

# Define your Google Cloud project ID
# This needs to be defined before initializing the BigQuery client
project_id = "capstone-467505" # Replace with your actual project ID

try:
    # Check if the bigquery client object exists and is of the correct type
    bigquery_client = bigquery.Client(project=project_id) # Re-initialize BigQuery client here
except NameError:
    print("BigQuery client not found. Please ensure you have authenticated with Google Cloud and initialized the BigQuery client.")
    # You might want to add authentication/client initialization here if it's not handled in a previous cell
    # Example:
    # auth.authenticate_user()
    # project_id = "your-google-cloud-project-id" # Replace with your project ID
    # bigquery_client = bigquery.Client(project=project_id)
    raise # Re-raise the error if the client is not available
except Exception as e:
     print(f"An error occurred while initializing BigQuery client: {e}")
     raise


# Define your BigQuery dataset and table names
dataset_id = "CRM_Capstone_Dataset" # Replace with your dataset ID
# dataset_id = "your_dataset_id" # Replace with your dataset ID

# Create the dataset if it doesn't exist
dataset_ref = bigquery_client.dataset(dataset_id)
try:
    bigquery_client.get_dataset(dataset_ref)
    print(f"Dataset '{dataset_id}' already exists.")
except Exception:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # Set the appropriate location for your dataset
    bigquery_client.create_dataset(dataset, timeout=30)
    print(f"Dataset '{dataset_id}' created.")


# Define job configuration for loading
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE") # Overwrite the table if it exists

for sheet_name, df in cleaned_data.items():
    table_id = f"{dataset_id}.{sheet_name}" # Table name will be the sheet name

    try:
        # Ensure column names are valid BigQuery identifiers (replace spaces/special chars)
        df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=False)

        # Upload DataFrame to BigQuery
        job = bigquery_client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )  # Make an API request

        job.result()  # Wait for the job to complete.

        print(f"✅ Uploaded {sheet_name} to BigQuery table: {table_id}")

    except Exception as e:
        print(f"❌ Error uploading data to BigQuery for {sheet_name}: {e}")
        print(f"DataFrame columns for {sheet_name}: {df.columns.tolist()}")


# ---------- Step 7: SQL Modeling — Correct Funnel & Joins (using SQLite connection) ----------
# Drop the view if it exists to ensure it's recreated with the latest definition
conn.execute("DROP VIEW IF EXISTS Funnel_Stages;")
print("✅ Dropped existing Funnel_Stages view (if it existed).")

funnel_view_sql = """
CREATE VIEW Funnel_Stages AS -- CREATE VIEW instead of CREATE VIEW IF NOT EXISTS
SELECT
    d.Deal_ID,
    l.Lead_Name,
    c.Company_Name,
    r.Region_Name,
    s.Rep_Name,
    d.Created_Date,
    d.Closed_Date,
    d.Total_Duration_Days AS Deal_Total_Duration_Days, -- Rename to avoid confusion
    sh.Stage,
    CASE
        WHEN UPPER(sh.Stage) LIKE '%LEAD CAPTURED%' THEN 'Lead Captured'
        WHEN UPPER(sh.Stage) LIKE '%QUALIFIED%' THEN 'Qualified'
        WHEN UPPER(sh.Stage) LIKE '%CONTACTED%' THEN 'Contacted'
        WHEN UPPER(sh.Stage) LIKE '%DEMO SCHEDULED%' THEN 'Demo Scheduled'
        WHEN UPPER(sh.Stage) LIKE '%DEMO COMPLETED%' THEN 'Demo Completed'
        WHEN UPPER(sh.Stage) LIKE '%DEMO%' THEN 'Demo'
        WHEN UPPER(sh.Stage) LIKE '%PROPOSAL SENT%' THEN 'Proposal Sent'
        WHEN UPPER(sh.Stage) LIKE '%PROPOSAL%' THEN 'Proposal'
        WHEN UPPER(sh.Stage) LIKE '%NEGOTIATION%' THEN 'Negotiation'
        WHEN UPPER(sh.Stage) LIKE '%DEAL WON%' THEN 'Deal Won'
        WHEN UPPER(sh.Stage) LIKE '%DEAL LOST%' THEN 'Deal Lost'
        ELSE 'Other'
    END AS Funnel_Stage,
    DATE(sh.Timestamp) AS Stage_Change_Date,
    sh."Stage_Engagement_in_Days" AS Stage_Duration_Days, -- Corrected column name here
    l.Rep_ID -- Include Rep_ID from Leads for easier joining with Sales_Reps if needed
FROM Deals d
LEFT JOIN Leads l ON d.Lead_ID = l.Lead_ID
LEFT JOIN Companies c ON d.Company_ID = c.Company_ID
LEFT JOIN Regions r ON l.Region_ID = r.Region_ID
LEFT JOIN Sales_Reps s ON l.Rep_ID = s.Rep_ID -- Join with Sales_Reps
LEFT JOIN Stage_History sh ON l.Lead_ID = sh.Lead_ID;
"""
conn.executescript(funnel_view_sql)
print("✅ SQL Funnel_Stages view created.")


# ---------- Step 8: Analysis Queries (using SQLite connection) ----------
queries = {
    "Funnel_Stage_Counts": """
        SELECT Funnel_Stage, COUNT(*) AS Deal_Count
        FROM Funnel_Stages
        GROUP BY Funnel_Stage
        ORDER BY Deal_Count DESC;
    """,
    "Avg_Duration_Per_Stage": """
        SELECT Funnel_Stage, ROUND(AVG(Stage_Duration_Days), 1) AS Avg_Days -- Use Stage_Duration_Days
        FROM Funnel_Stages
        WHERE Stage_Duration_Days IS NOT NULL
        GROUP BY Funnel_Stage
        ORDER BY Avg_Days DESC;
    """,
    "Win_Rate_By_Rep": """
        SELECT
            s.Rep_Name,
            ROUND(
              SUM(CASE WHEN fs.Funnel_Stage = 'Deal Won' THEN 1 ELSE 0 END) * 100.0
              / NULLIF(COUNT(DISTINCT fs.Deal_ID), 0), 1
            ) AS Win_Rate_Percent -- Calculate win rate based on distinct deals
        FROM Funnel_Stages fs
        LEFT JOIN Sales_Reps s ON fs.Rep_ID = s.Rep_ID
        WHERE fs.Funnel_Stage IN ('Deal Won', 'Deal Lost') -- Only consider closed deals for win rate
        GROUP BY s.Rep_Name
        ORDER BY Win_Rate_Percent DESC;
    """
}

result_dfs = {}
for name, sql in queries.items():
    try:
        result_dfs[name] = pd.read_sql_query(sql, conn)
        print(f"Successfully ran analysis query: {name}")
    except Exception as e:
        print(f"An error occurred while running query '{name}': {e}")


# ---------- Step 9: Load Analysis Results to BigQuery and Export to Excel/Google Sheets ----------
print("⬆️ Attempting to upload analysis results to BigQuery...")

# Re-using the existing BigQuery client and dataset_id
try:
    # Check if the bigquery client object exists
    bigquery_client
except NameError:
    print("BigQuery client not found. Please ensure you have authenticated with Google Cloud and initialized the BigQuery client.")
    raise

try:
    # Create the dataset if it doesn't exist (already done for main tables, but good to ensure)
    dataset_ref = bigquery_client.dataset(dataset_id)
    bigquery_client.get_dataset(dataset_ref) # Check if dataset exists
except Exception:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    bigquery_client.create_dataset(dataset, timeout=30)
    print(f"Dataset '{dataset_id}' created for analysis results.")


# Define job configuration for loading analysis results
job_config_analysis = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE") # Overwrite tables


for name, df in result_dfs.items():
    # Create table ID using the analysis name
    table_id_analysis = f"{dataset_id}.{name}"

    try:
        # Ensure column names are valid BigQuery identifiers
        df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=False)

        # Upload DataFrame to BigQuery
        job = bigquery_client.load_table_from_dataframe(
            df, table_id_analysis, job_config=job_config_analysis
        )

        job.result() # Wait for the job to complete

        print(f"✅ Uploaded analysis results '{name}' to BigQuery table: {table_id_analysis}")

    except Exception as e:
        print(f"❌ Error uploading analysis results to BigQuery for '{name}': {e}")
        print(f"DataFrame columns for '{name}': {df.columns.tolist()}")


# Export results to Excel and Google Sheets (existing code)
output_excel_path = "/content/drive/MyDrive/CAPSTONE_FINAL/CRM_Funnel_Stage_Results.xlsx"
with pd.ExcelWriter(output_excel_path, engine="xlsxwriter") as writer:
    for name, df in result_dfs.items():
        df.to_excel(writer, sheet_name=name, index=False)
        # Also write to Google Sheets
        try:
            worksheet = spreadsheet.worksheet(name)
        except gspread.WorksheetNotFound:
            worksheet = spreadsheet.add_worksheet(title=name, rows=df.shape[0], cols=df.shape[1])

        worksheet.clear()
        set_with_dataframe(worksheet, df)
        print(f"Successfully wrote analysis results to Google Sheet: {name}")


conn.close()
print(f"📊 Funnel stage results exported to {output_excel_path}, Google Sheets, and Google Drive CSVs.")
print("✅ You can also open the .sqlite DB in DB Browser and browse 'Funnel_Stages' to see details.")

Successfully loaded data from sheet: Leads
Successfully loaded data from sheet: Sales_Reps
Successfully loaded data from sheet: Regions
Successfully loaded data from sheet: Companies
Successfully loaded data from sheet: Stage_History
Successfully loaded data from sheet: Deals
✅ Data Cleaning Complete.
Successfully wrote cleaned data back to sheet: Leads
Successfully wrote cleaned data back to sheet: Sales_Reps
Successfully wrote cleaned data back to sheet: Regions
Successfully wrote cleaned data back to sheet: Companies
Successfully wrote cleaned data back to sheet: Stage_History
Successfully wrote cleaned data back to sheet: Deals
💾 Cleaned data written back to Google Sheet.
Successfully saved cleaned data to CSV in Google Drive: /content/drive/MyDrive/CAPSTONE_FINAL/Generated_Data/Leads_cleaned.csv
Successfully saved cleaned data to CSV in Google Drive: /content/drive/MyDrive/CAPSTONE_FINAL/Generated_Data/Sales_Reps_cleaned.csv
Successfully saved cleaned data to CSV in Google Drive: 